In [1]:
%env AWS_REQUEST_PAYER=requester

env: AWS_REQUEST_PAYER=requester


In [2]:
import numpy as np
from pystac_client import Client
import rasterio as rio
from rasterio.windows import Window
import boto3
from tqdm import tqdm
import tqdm.notebook as tq

In [3]:
import rasterio as rio
import xarray as xr
import rioxarray

def xarray_from_list(file_list, time_list, observation):
    datasets=[]

    p_bar=tq.tqdm(range(len(file_list)), position=0, leave=True)
    for i in range(0, len(file_list)):
        url = file_list[i]
        time_name = time_list[i]
        da = rioxarray.open_rasterio (url)
        da = da.squeeze().drop(labels='band')
        # Window the dataset using the coordinates (0, 0, 256, 256)
        windowed_da = da.isel(x=slice(2560, 2560+256), y=slice(2560, 2560+256))
        coords = windowed_da.coords
        coords['time'] = time_name
        windowed_da.assign_coords(coords)
        ds = windowed_da.to_dataset(name=observation)
        datasets.append(ds)
        p_bar.update(1)
        p_bar.refresh()
    DS = xr.concat(datasets, join='override', dim='time')
    return DS

In [4]:
h = 16
v = 5
region = 'CU'
#bands = ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22', 'qa_pixel']
#bands = ['diag', 'intr', 'intsm', 'inwam', 'mask', 'shade']
#bands = ['diag']
bands = ['intsm']

#cp = configparser.ConfigParser()
#cp.read(os.path.join(os.environ['HOME'],'.aws','credentials'))
#key = cp['default']['aws_access_key_id']
#secret = cp['default']['aws_secret_access_key']

stac = Client.open('https://landsatlook.usgs.gov/stac-server')

# datetime='1980-01-01/2024-01-01',

my_date_range ='2020-01-01/2024-01-01'
query = stac.search(collections=['landsat-c2l3-dswe'],
                    datetime=my_date_range,
                    query={'landsat:grid_horizontal': {'eq': f'{h:02}'},
                           'landsat:grid_vertical': {'eq': f'{v:02}'},
                           'landsat:grid_region': {'eq': region},
                           #'',
                           #'platform': {'eq': 'LANDSAT_8'},
                          }
                   )

stuff = query.get_all_items_as_dict()['features']
# dates = [pd.to_datetime(r['properties']['datetime'], utc=True) for r in stuff]


print(len(stuff))

688


In [5]:
stuff[0]['properties'].keys()

dict_keys(['start_datetime', 'end_datetime', 'datetime', 'platform', 'instruments', 'landsat:grid_horizontal', 'landsat:grid_vertical', 'landsat:grid_region', 'landsat:scene_count', 'eo:cloud_cover', 'landsat:cloud_shadow_cover', 'landsat:snow_ice_cover', 'landsat:fill', 'proj:epsg', 'proj:shape', 'proj:transform', 'created', 'updated'])

In [6]:
file_list = []
time_list = []
for item in stuff:
    in_path = item['assets']['intsm']['alternate']['s3']['href']
    file_list.append(in_path)
    dt = item['properties']['datetime']
    time_list.append(dt)

In [7]:
observation = 'dswe_intsm'
my_ds = xarray_from_list(file_list, time_list, observation)

  0%|          | 0/688 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
my_ds

In [ ]:
my_ds['dswe_intsm']

In [ ]:
my_ds['dswe_intsm'][0]

In [ ]:
my_ds['dswe_intsm'][0].values.min()

In [ ]:
my_ds['dswe_intsm'][0].values.max()

In [ ]:
my_ds['dswe_intsm'][0].values.mean()

In [ ]:
ary = my_ds['dswe_intsm'][0].values

In [ ]:
type(ary)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(ary, cmap='prism')

In [ ]:
import numpy.ma as ma

In [ ]:
# Specify the condition for nodata values
nodata_condition = (ary == 255)

# Create a masked array
masked_array = ma.masked_array(ary, nodata_condition)

In [ ]:
plt.imshow(masked_array, cmap='prism')

In [ ]:
# Plot histograms
plt.hist(ary, bins=50, alpha=0.5, label='Original Array')
plt.hist(masked_array, bins=50, alpha=0.5, label='Masked Array')

plt.title('Histogram of Original and Masked Arrays')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
plt.hist(ary, bins=4, alpha=0.5, label='Original Array')

In [ ]:
plt.show()

In [ ]:
ary.shape

In [ ]:
plt.hist(masked_array, bins=40, alpha=0.5, label='Original Array')

In [ ]:
from rasterio.plot import show_hist

In [ ]:
show_hist(masked_array, bins=4, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title="Histogram")

In [ ]:
256 * 256